# Task 1 of Assignment 2

**Objective:** Process the MIMIC chest X-ray dataset as a single entity, training all pathologies together.

**Split Ratio:** 5:3:2 (50% training, 30% validation, 20% testing)

**Pathology Grouping:** No grouping, all 13 pathologies (Atelectasis, Cardiomegaly, etc.) treated uniformly

**Key Features:**
- Data split with fixed seed (seed=42) for consistency
- NaN values converted to 0, no filtering of all-NaN samples
- Outputs three datasets: train_dataset, val_dataset, test_dataset

In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!unzip "/content/drive/MyDrive/sph6004-2025/data/generalized-image-embeddings-for-the-mimic-chest-x-ray-dataset-1.0.zip"

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: generalized-image-embeddings-for-the-mimic-chest-x-ray-dataset-1.0/files/p19/p19796330/s51132520/a9e8e82e-d90e58a7-f28de941-f1e980a8-2d71737a.tfrecord  
  inflating: generalized-image-embeddings-for-the-mimic-chest-x-ray-dataset-1.0/files/p19/p19796330/s53980046/f96a7c3b-743943f1-f33e2e08-7325391b-3db36721.tfrecord  
  inflating: generalized-image-embeddings-for-the-mimic-chest-x-ray-dataset-1.0/files/p19/p19796474/s52981359/d298b8d4-f51825fb-bb351c1e-46f32718-8be91e1f.tfrecord  
  inflating: generalized-image-embeddings-for-the-mimic-chest-x-ray-dataset-1.0/files/p19/p19796562/s59195152/665a0427-215b7179-410d67a5-dd6b2252-90c7075a.tfrecord  
  inflating: generalized-image-embeddings-for-the-mimic-chest-x-ray-dataset-1.0/files/p19/p19796678/s54648930/0f3b6201-fb94a849-0af7fdcb-eeba98c8-2e6aa992.tfrecord  
  inflating: generalized-image-embeddings-for-the-mimic-chest-x-ray-dataset-1.0/files/p19/p19796796/s57422647/37c6435c-cad30869-447b882a-c3fc05

### 1. Load Data (All Label)

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve
from tqdm import tqdm
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set TensorFlow logging level to suppress unnecessary output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')

# Helper function: Load embeddings from TFRecord files
def load_embedding(embedding_path):
    raw_dataset = tf.data.TFRecordDataset([embedding_path])
    for raw_record in raw_dataset.take(1):
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        embedding_feature = example.features.feature['embedding']
        embedding_values = embedding_feature.float_list.value
    embedding = torch.tensor(embedding_values)
    if torch.isnan(embedding).any() or torch.isinf(embedding).any():
        raise ValueError(f"NaN or Inf found in embedding: {embedding_path}")
    return embedding

# Custom dataset class for MIMIC chest X-ray embeddings
class MIMIC_Embed_Dataset(Dataset):
    pathologies = [
        "Enlarged Cardiomediastinum", "Cardiomegaly", "Lung Opacity", "Lung Lesion",
        "Edema", "Consolidation", "Pneumonia", "Atelectasis", "Pneumothorax",
        "Pleural Effusion", "Pleural Other", "Fracture", "Support Devices"
    ]

    def __init__(self, embedpath, csvpath, metacsvpath, views=["PA"], mode="train", seed=42, split_ratio=[0.5, 0.3, 0.2], unique_patients=True):
        super().__init__()
        np.random.seed(seed)  # Set seed for reproducibility
        self.pathologies = sorted(self.pathologies)
        self.mode = mode
        self.embedpath = embedpath
        self.csv = pd.read_csv(csvpath)
        self.metacsv = pd.read_csv(metacsvpath)
        self.split_ratio = split_ratio

        # Merge CSV data
        self.csv = self.csv.set_index(["subject_id", "study_id"])
        self.metacsv = self.metacsv.set_index(["subject_id", "study_id"])
        self.csv = self.csv.join(self.metacsv).reset_index()

        self.csv["view"] = self.csv["ViewPosition"]
        self.limit_to_selected_views(views)

        if unique_patients:
            self.csv = self.csv.groupby("subject_id").first().reset_index()

        # Split dataset into train, validation, and test sets
        n_row = self.csv.shape[0]
        if self.mode == "train":
            self.csv = self.csv[:int(n_row * self.split_ratio[0])]
        elif self.mode == "valid":
            self.csv = self.csv[int(n_row * self.split_ratio[0]):int(n_row * (self.split_ratio[0] + self.split_ratio[1]))]
        elif self.mode == "test":
            self.csv = self.csv[int(n_row * (self.split_ratio[0] + self.split_ratio[1])):]
        else:
            raise ValueError(f"mode must be one of [train, valid, test], got {self.mode}")

        # Process labels
        healthy = self.csv["No Finding"] == 1
        labels = []
        for pathology in self.pathologies:
            if pathology in self.csv.columns:
                self.csv.loc[healthy, pathology] = 0
                mask = self.csv[pathology]
            labels.append(mask.values)
        self.labels = np.asarray(labels).T.astype(np.float32)
        self.labels[np.isnan(self.labels)] = 0  # Replace all NaN values with 0
        self.labels = np.where((self.labels != 0) & (self.labels != 1), 0, self.labels)

        self.csv = self.csv.reset_index(drop=True)
        self.pathologies = list(np.char.replace(self.pathologies, "Pleural Effusion", "Effusion"))
        self.csv["offset_day_int"] = self.csv["StudyDate"]
        self.csv["patientid"] = self.csv["subject_id"].astype(str)

    def limit_to_selected_views(self, views):
        # Limit dataset to specified views
        if not isinstance(views, list):
            views = [views]
        if '*' in views:
            views = ["*"]
        self.views = views
        self.csv["view"] = self.csv["view"].fillna("UNKNOWN")
        if "*" not in views:
            self.csv = self.csv[self.csv["view"].isin(self.views)]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Retrieve a single sample from the dataset
        sample = {}
        sample["idx"] = idx
        sample["lab"] = self.labels[idx]
        subjectid = str(self.csv.iloc[idx]["subject_id"])
        studyid = str(self.csv.iloc[idx]["study_id"])
        dicom_id = str(self.csv.iloc[idx]["dicom_id"])
        embed_file = os.path.join(self.embedpath, f"p{subjectid[:2]}", f"p{subjectid}", f"s{studyid}", f"{dicom_id}.tfrecord")
        sample["embedding"] = load_embedding(embed_file)
        return sample

# Function to initialize datasets with given paths, seed, and split ratio
def initialize_mimic_datasets(embedpath, csvpath, metacsvpath, seed=42, split_ratio=[0.5, 0.3, 0.2]):
    """
    Initialize MIMIC chest X-ray datasets with specified paths, seed, and split ratio.

    Args:
        embedpath (str): Path to the embedding files
        csvpath (str): Path to the CheXpert CSV file
        metacsvpath (str): Path to the metadata CSV file
        seed (int): Random seed for reproducibility (default: 42)
        split_ratio (list): Train, validation, test split ratio (default: [0.5, 0.3, 0.2])

    Returns:
        tuple: (train_dataset, val_dataset, test_dataset)
    """
    train_dataset = MIMIC_Embed_Dataset(
        embedpath=embedpath,
        csvpath=csvpath,
        metacsvpath=metacsvpath,
        mode="train",
        seed=seed,
        split_ratio=split_ratio
    )
    val_dataset = MIMIC_Embed_Dataset(
        embedpath=embedpath,
        csvpath=csvpath,
        metacsvpath=metacsvpath,
        mode="valid",
        seed=seed,
        split_ratio=split_ratio
    )
    test_dataset = MIMIC_Embed_Dataset(
        embedpath=embedpath,
        csvpath=csvpath,
        metacsvpath=metacsvpath,
        mode="test",
        seed=seed,
        split_ratio=split_ratio
    )
    return train_dataset, val_dataset, test_dataset

# Data paths
embedpath = "/content/generalized-image-embeddings-for-the-mimic-chest-x-ray-dataset-1.0/files"
csvpath = "/content/drive/MyDrive/sph6004-2025/data/mimic-cxr-2.0.0-chexpert.csv"
metacsvpath = "/content/drive/MyDrive/sph6004-2025/data/mimic-cxr-2.0.0-metadata.csv"

train_dataset, val_dataset, test_dataset = initialize_mimic_datasets(
    embedpath=embedpath,
    csvpath=csvpath,
    metacsvpath=metacsvpath,
    seed=42,
    split_ratio=[0.5, 0.3, 0.2]
)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import joblib
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier, ClassifierChain
from sklearn.metrics import precision_recall_curve, f1_score
import numpy as np
import torch
from tqdm import tqdm

# Initialize the dataset and handle missing values
train_dataset = MIMIC_Embed_Dataset(embedpath, csvpath, metacsvpath, mode="train")
val_dataset = MIMIC_Embed_Dataset(embedpath, csvpath, metacsvpath, mode="valid")
test_dataset = MIMIC_Embed_Dataset(embedpath, csvpath, metacsvpath, mode="test")

# Handle missing labels by setting NaNs to 0
train_dataset.labels[np.isnan(train_dataset.labels)] = 0
val_dataset.labels[np.isnan(val_dataset.labels)] = 0
test_dataset.labels[np.isnan(test_dataset.labels)] = 0

# Get the embeddings and labels
X_train = np.array([train_dataset[idx]['embedding'].numpy() for idx in range(len(train_dataset))])
y_train = train_dataset.labels
X_val = np.array([val_dataset[idx]['embedding'].numpy() for idx in range(len(val_dataset))])
y_val = val_dataset.labels
X_test = np.array([test_dataset[idx]['embedding'].numpy() for idx in range(len(test_dataset))])
y_test = test_dataset.labels

# Standardize the features (embeddings)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Define models to train
model_dict = {
    'rf_model': RandomForestClassifier(
        random_state=42,
        n_jobs=2,
        class_weight='balanced',
        n_estimators=200,
        max_depth=15
  )
}

# wrapped in a multi-output classifier
ovr_models = ['rf_model']

for model_name, model in model_dict.items():
    print(f"\n{'=' * 40}")
    print(f"{model_name}")
    print(f"{'=' * 40}\n")

    if model_name in ovr_models:
        model = MultiOutputClassifier(model, n_jobs=2)
#
    cv = KFold(n_splits=5, shuffle=True, random_state=42)

    cross_val_scores = cross_val_score(
        model, X_train, y_train, cv=cv,
        scoring=lambda est, X, y: roc_auc_score(
            y,
            np.column_stack([p[:, 1] for p in est.predict_proba(X)]),
            average='macro',
            multi_class='ovo'
        ),
        n_jobs=5
    )
    print(f"{model_name} cross-validation AUC: {cross_val_scores.mean():.4f}")


    # Train model
    model.fit(X_train, y_train)
    joblib.dump(model, f'{model_name}.pkl')

    y_val_pred_proba = model.predict_proba(X_val)
    y_val_score = np.column_stack([p[:, 1] for p in y_val_pred_proba])  # shape: [n_samples, n_labels]

    val_auc = roc_auc_score(y_val, y_val_score, average='macro')
    print(f"Validation AUC: {val_auc:.4f}")

    optimal_val_thresholds = []

    for i in range(y_val.shape[1]):
        y_true = y_val[:, i]
        y_prob = y_val_score[:, i]
        precision, recall, thresholds = precision_recall_curve(y_true, y_prob)
        f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
        best_threshold = thresholds[np.argmax(f1)]
        optimal_val_thresholds.append(best_threshold)

    y_val_pred = np.zeros_like(y_val)

    for i in range(y_val.shape[1]):
        y_val_pred[:, i] = (y_val_score[:, i] > optimal_val_thresholds[i]).astype(int)

    print("\nValidation Classification Report:")
    print(classification_report(y_val, y_val_pred, digits=2))

    # Test
    y_pred_proba = model.predict_proba(X_test)

    optimal_thresholds = []
    for i in range(y_test.shape[1]):
      y_true = y_test[:, i]
      y_score = y_pred_proba[i][:, 1]
      precision, recall, thresholds = precision_recall_curve(y_true, y_score)
      f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
      best_threshold = thresholds[np.argmax(f1)]
      optimal_thresholds.append(best_threshold)

    y_pred = np.zeros_like(y_test)
    for i in range(y_test.shape[1]):
      y_pred[:, i] = (y_pred_proba[i][:, 1] > optimal_thresholds[i]).astype(int)
    print("\nTest Classification Report:")
    print(classification_report(y_test, y_pred, digits=2))

'''
    test_auc = roc_auc_score(
        y_test,
        np.column_stack([p[:, 1] for p in y_pred_proba]),
        average='macro',
        multi_class='ovo'
    )
    print(f"testset AUC: {test_auc:.4f}")

    y_pred = (np.column_stack([p[:, 1] for p in y_pred_proba]) > 0.3).astype(int)
    print(classification_report(y_test, model.predict(X_test)))
    '''


rf_model

rf_model cross-validation AUC: 0.8440
Validation AUC: 0.8450

Validation Classification Report:
              precision    recall  f1-score   support

           0       0.36      0.57      0.44      1211
           1       0.37      0.53      0.44      1085
           2       0.15      0.32      0.20       243
           3       0.30      0.43      0.36       356
           4       0.08      0.17      0.11       189
           5       0.11      0.20      0.14       292
           6       0.23      0.29      0.26       392
           7       0.36      0.56      0.44      1612
           8       0.57      0.63      0.60      1121
           9       0.10      0.39      0.16       123
          10       0.18      0.45      0.26       685
          11       0.23      0.32      0.27       175
          12       0.39      0.40      0.39       572

   micro avg       0.31      0.49      0.38      8056
   macro avg       0.26      0.40      0.31      8056
weighted avg       0.34    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Test Classification Report:
              precision    recall  f1-score   support

           0       0.36      0.57      0.44       855
           1       0.39      0.55      0.45       761
           2       0.16      0.23      0.19       166
           3       0.31      0.48      0.38       229
           4       0.08      0.28      0.13       139
           5       0.12      0.14      0.13       184
           6       0.17      0.41      0.24       271
           7       0.33      0.66      0.44      1120
           8       0.60      0.63      0.61       742
           9       0.09      0.20      0.13        81
          10       0.28      0.32      0.30       431
          11       0.20      0.23      0.21       105
          12       0.30      0.56      0.39       398

   micro avg       0.32      0.52      0.39      5482
   macro avg       0.26      0.40      0.31      5482
weighted avg       0.34      0.52      0.40      5482
 samples avg       0.09      0.13      0.10      54

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


'\n    test_auc = roc_auc_score(\n        y_test,\n        np.column_stack([p[:, 1] for p in y_pred_proba]),\n        average=\'macro\',\n        multi_class=\'ovo\'\n    )\n    print(f"testset AUC: {test_auc:.4f}")\n\n    y_pred = (np.column_stack([p[:, 1] for p in y_pred_proba]) > 0.3).astype(int)\n    print(classification_report(y_test, model.predict(X_test)))\n    '